In [17]:
import pandas as pd
import numpy as np
df = pd.read_excel("../데이터자료/join_data.xlsx")
df.columns
df = df[["DateTime", "종가_ex",'1Y_Mid_irs','10Y_Mid_irs','1Y_Mid_crs','국고1년','통안364일' ,'Mid_ndf','M1_스왑포인트']]
df["M1_스왑포인트"] = df["M1_스왑포인트"]/100
df.head()
df = df.set_index("DateTime")

In [18]:
df["1Y_Mid_irs대비"] =df["1Y_Mid_irs"]-df["1Y_Mid_irs"].shift(1)
df["10Y_Mid_irs대비"] =df["10Y_Mid_irs"]-df["10Y_Mid_irs"].shift(1)
df["1Y_Mid_crs대비"] =df["1Y_Mid_crs"]-df["1Y_Mid_crs"].shift(1)
df["국고1년대비"] =df["국고1년"]-df["국고1년"].shift(1)
df["통안364일대비"] = df["통안364일"] - df["통안364일"].shift(1)
df["Mid_ndf대비"] = df["Mid_ndf"] - df["Mid_ndf"].shift(1)
df["M1_스왑포인트대비"] = df["M1_스왑포인트"]- df["M1_스왑포인트"].shift(1)
df.head()

,종가_ex,1Y_Mid_irs,10Y_Mid_irs,1Y_Mid_crs,국고1년,통안364일,Mid_ndf,M1_스왑포인트,1Y_Mid_irs대비,10Y_Mid_irs대비,1Y_Mid_crs대비,국고1년대비,통안364일대비,Mid_ndf대비,M1_스왑포인트대비
DateTime,,,,,,,,,,,,,,,
2012-08-01,1126.5,2.85,2.91,2.09,2.82,2.80,1132.25,2.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012-08-02,1131.7,2.82,2.86,2.08,2.79,2.77,1134.00,2.5,-0.03,-0.05,-0.01,-0.03,-0.03,1.75,-0.1
2012-08-03,1134.8,2.79,2.84,2.07,2.77,2.75,1138.00,2.4,-0.03,-0.02,-0.01,-0.02,-0.02,4.00,-0.1
2012-08-06,1129.0,2.81,2.85,2.07,2.78,2.76,1128.50,2.4,0.02,0.01,0.00,0.01,0.01,-9.50,0.0
2012-08-07,1128.8,2.82,2.87,2.09,2.77,2.75,1129.00,2.5,0.01,0.02,0.02,-0.01,-0.01,0.50,0.1


In [19]:
df["전날 종가_ex"] = df["종가_ex"].shift(1)
df = df.dropna()
df.columns


Index(['종가_ex', '1Y_Mid_irs', '10Y_Mid_irs', '1Y_Mid_crs', '국고1년', '통안364일',
       'Mid_ndf', 'M1_스왑포인트', '1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비',
       '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트대비', '전날 종가_ex'],
      dtype='object')

In [20]:
# 종속변수 독립변수 설정
df_Target = df["종가_ex"]
df_Data = df[['1Y_Mid_irs대비', '10Y_Mid_irs대비','1Y_Mid_crs대비', '국고1년대비', '통안364일대비', 'Mid_ndf대비', 'M1_스왑포인트','전날 종가_ex']]
df_Data.feature= df_Data.columns
df_Data.feature


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_20248\3284639614.py:4: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_Mid_irs대비', '10Y_Mid_irs대비', '1Y_Mid_crs대비', '국고1년대비', '통안364일대비',
       'Mid_ndf대비', 'M1_스왑포인트', '전날 종가_ex'],
      dtype='object')

In [21]:
# 스케일링 코드
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()

scaler.fit(df_Data)
df_Data_scaled= scaler.transform(df_Data)
df_scaled = pd.DataFrame (data = df_Data_scaled, columns=df_Data.feature)
df_scaled.index = df_Target.index

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 3.158e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        21:14:19   Log-Likelihood:                -7660.3
No. Observations:                2459   AIC:                         1.534e+04
Df Residuals:                    2450   BIC:                         1.539e+04
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const          1134.8939      0.110   1.03e+04      0.000    1134.678    1135.110
1Y_Mid_irs대비      0.2269      0.174      1.306      0.192      -0.114       0.568
10Y_Mid_irs대비    -0.1432      0.178     -0.804      0.422      -0.493       0.206
1Y_Mid_crs대비     -1.5418      0.112    -13.788      0.000      -1.761      -1.322
국고1년대비            0.3921      0.123      3.197      0.001       0.152       0.632
통안364일대비          0.0100      0.111      0.089      0.929      -0.209       0.228
Mid_ndf대비         0.5590      0.110      5.066      0.000       0.343       0.775
M1_스왑포인트         -0.1415      0.119     -1.188      0.235      -0.375       0.092
전날 종가_ex         55.3382      0.120    462.666      0.000      55.104      55.573
==============================================================================
Omnibus:                       44.133   Durbin-Watson:                   2.121
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               87.644
Skew:                           0.009   Prob(JB):                     9.29e-20
Kurtosis:                       3.925   Cond. No.                         3.07
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [24]:
# p값높은 컬럼들 제외
df_Target = df["종가_ex"]
df_scaled = df_scaled[['1Y_Mid_crs대비', '국고1년대비', 'Mid_ndf대비','전날 종가_ex']]

In [25]:
import statsmodels.api as sm
feature_add_std = sm.add_constant(df_scaled, has_constant='add')
# sm OLS 적합
model = sm.OLS(df_Target, feature_add_std)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary() 


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 6.318e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        21:14:21   Log-Likelihood:                -7661.9
No. Observations:                2459   AIC:                         1.533e+04
Df Residuals:                    2454   BIC:                         1.536e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const         1134.8939      0.110   1.03e+04      0.000    1134.678    1135.110
1Y_Mid_crs대비    -1.5331      0.111    -13.759      0.000      -1.752      -1.315
국고1년대비           0.4101      0.111      3.678      0.000       0.191       0.629
Mid_ndf대비        0.5564      0.110      5.045      0.000       0.340       0.773
전날 종가_ex        55.3972      0.111    501.199      0.000      55.180      55.614
==============================================================================
Omnibus:                       45.348   Durbin-Watson:                   2.123
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.101
Skew:                           0.009   Prob(JB):                     1.65e-20
Kurtosis:                       3.943   Cond. No.                         1.18
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
X_train, X_test, y_train, y_test = train_test_split(feature_add_std,df_Target,test_size=0.2, random_state=16 )
lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse,mse

(5.1218787199673415, 26.233641622054297)

In [16]:
from sklearn.model_selection import cross_val_score
neg_mse_scores = cross_val_score(lr, feature_add_std, df_Target,scoring="neg_mean_squared_error", cv = 5)
rmse_scores = np.sqrt(-1 *neg_mse_scores)
avg_rmse = np.mean(rmse_scores)
avg_rmse,rmse_scores, neg_mse_scores

(8.131060132846235,
 array([ 4.0964659 ,  6.74277563,  5.72018172, 18.98432984,  5.11154757]),
 array([ -16.78103284,  -45.46502325,  -32.72047892, -360.40477941,
         -26.1279186 ]))

In [189]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_Data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 4.126e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        19:06:27   Log-Likelihood:                -7686.2
No. Observations:                2459   AIC:                         1.539e+04
Df Residuals:                    2452   BIC:                         1.543e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.8465      2.484      1.951      0.051      -0.024       9.717
1Y_베이시스대비     -0.2285      0.020    -11.669      0.000      -0.267      -0.190
국고1년대비         4.3814      6.411      0.683      0.494      -8.190      16.952
통안364일대비      -0.0433      1.219     -0.036      0.972      -2.435       2.348
Mid_ndf대비      0.0275      0.005      5.250      0.000       0.017       0.038
M1_스왑포인트      -0.1318      0.109     -1.210      0.226      -0.345       0.082
전날 종가_ex       0.9958      0.002    458.479      0.000       0.992       1.000
==============================================================================
Omnibus:                       54.914   Durbin-Watson:                   2.141
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              120.345
Skew:                           0.001   Prob(JB):                     7.37e-27
Kurtosis:                       4.084   Cond. No.                     6.55e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.55e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [190]:
df_Target = df["종가_ex"]
df_Data = df[['1Y_베이시스대비', 'Mid_ndf대비', '전날 종가_ex']]

In [191]:
import statsmodels.api as sm
feature_add = sm.add_constant(df_Data, has_constant='add')

# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 8.255e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        19:07:26   Log-Likelihood:                -7687.2
No. Observations:                2459   AIC:                         1.538e+04
Df Residuals:                    2455   BIC:                         1.541e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.5532      2.276      1.561      0.119      -0.911       8.017
1Y_베이시스대비     -0.2296      0.019    -11.789      0.000      -0.268      -0.191
Mid_ndf대비      0.0275      0.005      5.261      0.000       0.017       0.038
전날 종가_ex       0.9969      0.002    497.562      0.000       0.993       1.001
==============================================================================
Omnibus:                       54.965   Durbin-Watson:                   2.142
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              120.501
Skew:                          -0.004   Prob(JB):                     6.82e-27
Kurtosis:                       4.084   Cond. No.                     2.32e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.32e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [210]:
X_train, X_test, y_train, y_test = train_test_split(feature_add, df_Target,test_size=0.2, random_state=15 )

lr = LinearRegression()
lr.fit (X_train, y_train)
y_preds = lr.predict(X_test)
mse = mean_squared_error(y_test, y_preds)
rmse = np.sqrt(mse)
rmse

5.7593199385109415

In [196]:
df_Data.feature= df_Data.columns
df_Data.feature

C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_35636\2542687574.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df_Data.feature= df_Data.columns


Index(['1Y_베이시스대비', 'Mid_ndf대비', '전날 종가_ex'], dtype='object')

In [197]:
scaler = StandardScaler()
scaler.fit(df_Data)
df_Data_scaled= scaler.transform(df_Data)
df_scaled = pd.DataFrame (data = df_Data_scaled, columns=df_Data.feature)
df_scaled.index = df_Target.index
df_Data_scaled

array([[ 3.48741074e-01,  7.91666902e-02, -1.49840936e-01],
       [ 3.48741074e-01,  1.84972477e-01, -5.62318504e-02],
       [-3.50946270e-01, -4.49862243e-01, -4.26434230e-04],
       ...,
       [ 3.48741074e-01, -2.66390964e-02,  3.11027548e+00],
       [ 1.73819238e-01, -2.14738273e-01,  3.21288543e+00],
       [-7.00789942e-01, -4.85130838e-01,  2.90325538e+00]])

In [198]:
feature_add = sm.add_constant(df_scaled, has_constant='add')
# sm OLS 적합
model = sm.OLS(df_Target, feature_add)
fitted_model = model.fit()

# summary 함수통해 결과출력
fitted_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  종가_ex   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                 8.255e+04
Date:                Thu, 18 Aug 2022   Prob (F-statistic):               0.00
Time:                        19:13:43   Log-Likelihood:                -7687.2
No. Observations:                2459   AIC:                         1.538e+04
Df Residuals:                    2455   BIC:                         1.541e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       1134.8939      0.111   1.02e+04      0.000    1134.676    1135.112
1Y_베이시스대비     -1.3128      0.111    -11.789      0.000      -1.531      -1.094
Mid_ndf대비      0.5858      0.111      5.261      0.000       0.367       0.804
전날 종가_ex      55.3797      0.111    497.562      0.000      55.161      55.598
==============================================================================
Omnibus:                       54.965   Durbin-Watson:                   2.142
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              120.501
Skew:                          -0.004   Prob(JB):                     6.82e-27
Kurtosis:                       4.084   Cond. No.                         1.04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

TypeError: LinearRegression.fit() missing 1 required positional argument: 'y'

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_clf = DecisionTreeClassifier()
dt_clf.fit()
